In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import pandas as pd

In [0]:
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.3
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 100
# 5. Train Rate
train_rate = 0.9


In [0]:
#%% Data

# Data generation
df = pd.read_csv('/content/drive/My Drive/piv-project/pressure-data-csv.csv')

df = df.drop(labels='Unnamed: 0', axis='columns')

actual_df = df.copy()

for i in ['7','24','18','29','42','47','3']:
  df [i] = pd.DataFrame([0 for _ in range(df[i].size)])



Data = df.values #Returns Numpy Array
#val_Data = Data[1000:1500] # For Result Validation
Data = Data[1000:10000] # Sample Dataset

print(Data.shape)
# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim

# Normalization (0 to 1)
Min_Val = np.zeros(Dim)
Max_Val = np.zeros(Dim)

(9000, 50)


In [0]:
for i in range(Dim):
    Min_Val[i] = np.min(Data[:,i])
    Data[:,i] = Data[:,i] - np.min(Data[:,i])
    Max_Val[i] = np.max(Data[:,i])
    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    

#%% Missing introducing
p_miss_vec = p_miss * np.ones((Dim,1)) 
   
Missing = np.zeros((No,Dim))

for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    Missing[:,i] = 1.*B


In [0]:
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]
FindX = Data[range(1000),:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]
FindM = Missing[range(1000),:]

In [0]:
#%% Necessary Functions

# 1. Xavier Initialization Definition
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)
    
# Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C

In [0]:
'''
GAIN Consists of 3 Components
- Generator
- Discriminator
- Hint Mechanism
'''   
   
#%% GAIN Architecture   
   
#%% 1. Input Placeholders
# 1.1. Data Vector
X = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.2. Mask Vector 
M = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.3. Hint vector
H = tf.placeholder(tf.float32, shape = [None, Dim])
# 1.4. X with missing values
New_X = tf.placeholder(tf.float32, shape = [None, Dim])

#%% 2. Discriminator
D_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]))     # Data + Hint as inputs
D_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]))

D_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]))
D_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]))

D_W3 = tf.Variable(xavier_init([H_Dim2, Dim]))
D_b3 = tf.Variable(tf.zeros(shape = [Dim]))       # Output is multi-variate

theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

#%% 3. Generator
G_W1 = tf.Variable(xavier_init([Dim*2, H_Dim1]))     # Data + Mask as inputs (Random Noises are in Missing Components)
G_b1 = tf.Variable(tf.zeros(shape = [H_Dim1]))

G_W2 = tf.Variable(xavier_init([H_Dim1, H_Dim2]))
G_b2 = tf.Variable(tf.zeros(shape = [H_Dim2]))

G_W3 = tf.Variable(xavier_init([H_Dim2, Dim]))
G_b3 = tf.Variable(tf.zeros(shape = [Dim]))

theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

In [0]:
#%% GAIN Function

#%% 1. Generator
def generator(new_x,m):
    inputs = tf.concat(axis = 1, values = [new_x,m])  # Mask + Data Concatenate
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)   
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob
    
#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = tf.concat(axis = 1, values = [new_x,h])  # Hint + Data Concatenate
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

#%% 3. Other functions
# Random sample generator for Z
def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx


In [0]:
#%% Structure
# Generator
G_sample = generator(New_X,M)

# Combine with original data
Hat_New_X = New_X * M + G_sample * (1-M)

# Discriminator
D_prob = discriminator(Hat_New_X, H)

#%% Loss
D_loss1 = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) + (1-M) * tf.log(1. - D_prob + 1e-8)) 
G_loss1 = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))
MSE_train_loss = tf.reduce_mean((M * New_X - M * G_sample)**2) / tf.reduce_mean(M)

D_loss = D_loss1
G_loss = G_loss1 + alpha * MSE_train_loss 

#%% MSE Performance metric
MSE_test_loss = tf.reduce_mean(((1-M) * X - (1-M)*G_sample)**2) / tf.reduce_mean(1-M)

#%% Solver
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

# Sessions
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#%% Iterations

#%% Start Iterations
for it in tqdm(range(5000)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    _, D_loss_curr = sess.run([D_solver, D_loss1], feed_dict = {M: M_mb, New_X: New_X_mb, H: H_mb})
    _, G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = sess.run([G_solver, G_loss1, MSE_train_loss, MSE_test_loss],
                                                                       feed_dict = {X: X_mb, M: M_mb, New_X: New_X_mb, H: H_mb})
            
        
    #%% Intermediate Losses
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr)))
        print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr)))
        print()

  0%|          | 21/5000 [00:00<13:17,  6.25it/s]

Iter: 0
Train_loss: 0.3048
Test_loss: 0.3028



  2%|▏         | 120/5000 [00:00<02:32, 32.02it/s]

Iter: 100
Train_loss: 0.1112
Test_loss: 0.1133



  5%|▍         | 236/5000 [00:01<00:40, 118.58it/s]

...

 97%|█████████▋| 4835/5000 [00:25<00:00, 199.96it/s]

Iter: 4800
Train_loss: 0.04696
Test_loss: 0.06997



 99%|█████████▊| 4937/5000 [00:26<00:00, 194.43it/s]

Iter: 4900
Train_loss: 0.04823
Test_loss: 0.06877



100%|██████████| 5000/5000 [00:26<00:00, 187.82it/s]


In [0]:
#%% Final Loss
    
Z_mb = sample_Z(Test_No, Dim) 
M_mb = testM
X_mb = testX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
MSE_final, Sample = sess.run([MSE_test_loss, G_sample], feed_dict = {X: testX, M: testM, New_X: New_X_mb})
        
print('Final Test RMSE: ' + str(np.sqrt(MSE_final)))
print('MSE_final :'+ str(MSE_final))


Final Test RMSE: 0.06606924
MSE_final :0.004365144


In [0]:
Z_mb = sample_Z(1000, Dim) 
M_mb = FindM
X_mb = FindX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  

MSE_final, Sample = sess.run([MSE_test_loss,G_sample], feed_dict={X:FindX, M:FindM, New_X:New_X_mb})

In [0]:
print('MSE_final:',MSE_final)

MSE_final: 0.004385712


In [0]:
df_result = pd.DataFrame(Sample)

In [0]:
df_result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.820090,0.843513,0.000064,0.701050,0.634487,0.815988,0.000068,0.862313,0.746239,0.678159,0.819459,0.865113,0.891317,0.855594,0.794654,0.715432,0.766756,0.000025,0.903258,0.851522,0.719812,0.798933,0.747489,0.000020,0.808827,0.686500,0.803351,0.754369,0.000003,0.834011,0.715532,0.753040,0.729193,0.807312,0.815070,0.722472,0.729694,0.770726,0.734808,0.718324,0.642093,0.000009,0.847923,0.755742,0.727122,0.735410,0.000061,0.794741,0.777937,0.694893
1,0.823452,0.856092,0.000093,0.795144,0.741963,0.821993,0.000093,0.888440,0.822228,0.774060,0.824805,0.880366,0.904690,0.875927,0.798487,0.714896,0.782615,0.000029,0.848787,0.768369,0.725869,0.776938,0.697505,0.000025,0.693922,0.673797,0.755861,0.697287,0.000005,0.790397,0.721711,0.734393,0.713866,0.796328,0.821375,0.706102,0.729700,0.808201,0.819948,0.828490,0.633798,0.000012,0.853833,0.777414,0.778374,0.710593,0.000085,0.777219,0.770541,0.686325
2,0.883087,0.905930,0.000059,0.798744,0.727814,0.864398,0.000052,0.918069,0.836872,0.764872,0.850504,0.895065,0.915729,0.895866,0.830004,0.673789,0.751432,0.000015,0.905122,0.840408,0.652387,0.786047,0.784360,0.000011,0.850554,0.675807,0.819227,0.834311,0.000001,0.885547,0.704009,0.718564,0.763122,0.873767,0.878303,0.658788,0.677862,0.744310,0.790042,0.792120,0.633979,0.000006,0.823872,0.773836,0.744973,0.671797,0.000065,0.777925,0.780040,0.730865
3,0.862252,0.874077,0.000266,0.745865,0.686151,0.842486,0.000264,0.871320,0.763996,0.704912,0.847493,0.876687,0.868806,0.796248,0.721717,0.745354,0.759850,0.000083,0.752837,0.639869,0.761839,0.818367,0.735587,0.000052,0.672589,0.749882,0.839821,0.794948,0.000014,0.741363,0.695057,0.682268,0.714793,0.793743,0.770496,0.626791,0.682603,0.747304,0.760629,0.754771,0.620511,0.000040,0.822608,0.759370,0.727194,0.658631,0.000233,0.781292,0.789232,0.708639
4,0.783630,0.818613,0.000093,0.758755,0.705046,0.772416,0.000091,0.850510,0.790899,0.743137,0.791865,0.838946,0.870163,0.850621,0.790261,0.706429,0.755177,0.000046,0.849781,0.784521,0.733623,0.791264,0.731541,0.000038,0.758917,0.702330,0.784153,0.727406,0.000006,0.806643,0.681056,0.671948,0.662231,0.779493,0.797918,0.636165,0.649217,0.709718,0.747294,0.757286,0.598484,0.000016,0.810123,0.751295,0.735282,0.669242,0.000094,0.770103,0.758730,0.690198


In [0]:
actual_df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,-0.39325,-0.58336,-0.47175,-0.51374,-0.49806,-0.38474,-0.50544,-0.45865,-0.48011,-0.49821,-0.44725,-0.53410,-0.54686,-0.48339,-0.49871,-0.47177,-0.52128,-0.57743,-0.48199,-0.57963,-0.47891,-0.50790,-0.44786,-0.40451,-0.33950,-0.54578,-0.43843,-0.50953,-0.27542,-0.21293,-0.83023,-0.56492,-0.33589,-0.33840,-0.27590,-1.07506,-0.28629,-0.21245,-0.19798,-0.21598,-0.88088,-0.53669,-0.30754,-0.29569,-0.28139,-0.73443,-0.60668,-0.37522,-0.24987,-0.22637
1,-0.46394,-0.55902,-0.50768,-0.56086,-0.59995,-0.43636,-0.51445,-0.49466,-0.51900,-0.61017,-0.47843,-0.51443,-0.55461,-0.52448,-0.55597,-0.45817,-0.53070,-0.58489,-0.48573,-0.57592,-0.47383,-0.48221,-0.44337,-0.39221,-0.32945,-0.55203,-0.45594,-0.47436,-0.29060,-0.21293,-0.73171,-0.57603,-0.35809,-0.32698,-0.26483,-1.11513,-0.22032,-0.23385,-0.20568,-0.20430,-0.96510,-0.58501,-0.26755,-0.24605,-0.23469,-0.75008,-0.67076,-0.38356,-0.27873,-0.23883
2,-0.45185,-0.54247,-0.53390,-0.63754,-0.64403,-0.45964,-0.49543,-0.51646,-0.58449,-0.67248,-0.49261,-0.51050,-0.57957,-0.57584,-0.63996,-0.48083,-0.51500,-0.57246,-0.51376,-0.60658,-0.50838,-0.48418,-0.43350,-0.40890,-0.32670,-0.58329,-0.46761,-0.39597,-0.30375,-0.20317,-0.69556,-0.55380,-0.38129,-0.33225,-0.24637,-0.98382,-0.10461,-0.26677,-0.25956,-0.23306,-1.02326,-0.57026,-0.23357,-0.23214,-0.22352,-0.76885,-0.67498,-0.42529,-0.30086,-0.25460
3,-0.46859,-0.50743,-0.52905,-0.65972,-0.69890,-0.44648,-0.47741,-0.52404,-0.62338,-0.70363,-0.51151,-0.49673,-0.56838,-0.59639,-0.68578,-0.44729,-0.48046,-0.56749,-0.52871,-0.61494,-0.53075,-0.48517,-0.43799,-0.41066,-0.32122,-0.56766,-0.45789,-0.34774,-0.30982,-0.21683,-0.67477,-0.56399,-0.42264,-0.33752,-0.25744,-0.86956,-0.17824,-0.31203,-0.26555,-0.26092,-0.97914,-0.51457,-0.18858,-0.17257,-0.17073,-0.72087,-0.64547,-0.44115,-0.31144,-0.25709
4,-0.46208,-0.47238,-0.49991,-0.64955,-0.73710,-0.47887,-0.44337,-0.53730,-0.63566,-0.70752,-0.49828,-0.48787,-0.53740,-0.60853,-0.67241,-0.38744,-0.44122,-0.57495,-0.55207,-0.61773,-0.50330,-0.46837,-0.45055,-0.42120,-0.29745,-0.59892,-0.46956,-0.30653,-0.30578,-0.22561,-0.59704,-0.58066,-0.45089,-0.32610,-0.27405,-0.70328,-0.31976,-0.33919,-0.27239,-0.29418,-0.93603,-0.43104,-0.15260,-0.15370,-0.17073,-0.71148,-0.60162,-0.45283,-0.33261,-0.25793


In [0]:
df_result.to_excel('/content/drive/My Drive/piv-project/GAIN-Pressure-Result.xlsx', header=False)